In [1]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
            np.save(output_file_path, img_array)

input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
output_folder_path = "/kaggle/working/train"

jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [2]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            folder = input_folder.split("/")[-1]
            output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
            np.save(output_file_path, img_array)


jpg_to_npy_in_folders("/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [3]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

5121

In [4]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 52 2560 1792


In [5]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
2    2560
3    1792
0     717
1      52
Name: count, dtype: int64

In [6]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [7]:
from imblearn.over_sampling import RandomOverSampler
image_shape = images[0].shape
print(image_shape)
ros = RandomOverSampler(random_state=42)
flattened_images = [image.flatten() for image in images]
X_res, y_res = ros.fit_resample(flattened_images, np.array(df["Class"]))
images_over = np.reshape(X_res, (-1, *image_shape))
images_over.shape

(208, 176, 3)


(10240, 208, 176, 3)

In [8]:
images_over

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        ...,

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0,

In [9]:
images_over.shape

(10240, 208, 176, 3)

In [10]:
!rm -rf /kaggle/working/train

In [11]:
import os

names = {0:"milDem", 1:"moderateDim", 2:"nonDem", 3:"veryMildDem"}

os.makedirs("/kaggle/working/train/", exist_ok=True)

for i in range(len(y_res)):
    np.save(f"/kaggle/working/train/{names[y_res[i]]}{i}.npy", images_over[i])